# Homegrown Linear Regression (LR) via numpy

Here, we will implement linear regression (a single layer network, with an architecture "2-1", corresponding to a model with 2 inputs, and a single output node) using array programming, and also in network fashion using numpy.

Numpy provides an n-dimensional array class, and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; but it does not know anything about computation graphs, or deep learning, or gradients. However we can easily use numpy to fit a single-layer network (linear regression) to random data by manually implementing the forward and backward (gradient descent) passes through the network using numpy operations. The following code shows how to accomplish this:


The below code trains a linear regression model from random data (so the model learns nothing really; it is just an exercise).


**QUESTION** :Run the below code and modify as needed and answer the following question. What is the value of the gradient for the first iteration of gradient descent after running the above code?   

In [ ]:
import numpy as np

# m_rows is batch size; 
# D_in is input dimension;
# D_out is output dimension.
m_rows, D_in, D_out = 64, 2, 1
#m_rows, D_in, D_out = 64, 1000, 1

np.random.seed(seed=42) #fix the seed
# Create random input and output data
X_train = np.random.randn(m_rows, D_in)
y_train = np.random.randn(m_rows, D_out)

# Randomly initialize weights
W1 = np.random.randn(D_in, D_out) #[w0, w1, w2, ...w999]

learning_rate = 1e-3
for epoch in range(10):  #Gradient descent
    # Forward pass: compute predicted y
    y_pred = X_train.dot(W1)
    #y_pred = X_train @ W1 

    # Compute and print loss
    loss = np.square(y_pred - y_train).mean()
    print(f"Epoch:{epoch}, MSE: {loss}")

    # Backprop to compute gradients of w1 and w2 with respect to loss
    err = 2.0 * (y_pred - y_train)
    grad_W1 = X_train.T.dot(err)/m_rows  #weighted sum of the train data
    print(f"Epoch:{epoch}, Gradient: {grad_W1}")
    
    # Update weights via gradient descent
    W1 -= learning_rate * grad_W1
    print('--------------------')
    
print(f"Weights after training: ,{np.round(W1, 3)}")  

# Linear regression using tensors and autograd


Traditionally, say using Numpy, we had to manually implement both the forward and backward passes to train a  linear regression model (aka a single-layered neural network). Manually implementing the backward pass is not a big deal for a linear regression model or for a small/shallow network, but this can quickly get very tricky for larger multilayers networks.

Thankfully, we can use automatic differentiation to automate the computation of backward passes in neural networks. The autograd package in PyTorch provides exactly this functionality.

When using autograd,
* **the forward pass of your network (code for making a prediction) will define a computational graph;**
* nodes in the graph will be Tensors,
* and edges will be functions that produce output Tensors from input Tensors.

PyTorch builds up a graph as you compute the forward pass through the network, and one call to **backward()** on some result node (loss function) then augments each intermediate node in the graph with the gradient of the result node with respect to that intermediate node.

This sounds complicated, but it’s pretty simple to use in practice. We wrap our PyTorch Tensors in Variable objects; a Variable represents a node in a computational graph. If x is a Variable then x.data is a Tensor, and x.grad is another Variable holding the gradient of x with respect to some scalar value.

PyTorch Variables are PyTorch Tensors variables.

Here we use PyTorch tensor variables and autograd to implement our single-layer network (linear regression model); now we no longer need to manually implement the backward pass through the network:

**Question** Run the below code and modify as needed to get the value of the first element of the learnt linear regression model. What is the value of the first element of the learnt linear regression model?

In [ ]:
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# D_out is output dimension.
m_rows, D_in, D_out = 64, 1000, 1

torch.manual_seed(0)
# Create random Tensors to hold input and outputs, and wrap them in Variables.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Variables during the backward pass.
X_train = torch.randn((m_rows, D_in), requires_grad=False) #use (m_rows, D_in) or m_rows, D_in
Y_train = torch.randn(m_rows, D_out, requires_grad=False)

# Create random Tensors for weights, and wrap them in Variables.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Variables during the backward pass.
W1 = torch.randn(D_in, D_out,  requires_grad=True)

learning_rate = 1e-6
for epoch in range(50):
    # Forward pass: compute predicted y using operations on Variables; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    Y_pred = X_train.matmul(W1)

    # Compute and print loss using operations on Variables.
    # Now loss is a Variable of shape (1,) and loss.data is a Tensor of shape
    # (1,); loss.data[0] is a scalar value holding the loss.
    loss = (Y_pred - Y_train).pow(2).mean()
    if (epoch +1 )%10 ==0:  #print every 10 epochs
        print(f"Epoch:{epoch+1}, MSE: {loss.data.numpy():.6}")

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    # After this call W1.grad  will be Variables holding the gradient
    # of the loss with respect to W1.
    loss.backward()

    # Update weights using gradient descent; W1.data and W2.data are Tensors,
    # W1.grad are Variables and W1.grad.data aare
    # Tensors.
    W1.data -= learning_rate * W1.grad.data
    # Manually zero the gradients after updating weights
    W1.grad.data.zero_()

print('------------------------')    
# first value of the learnt linear regression model
print(...)

# PyTorch: optim


Up to this point we have updated the weights of our models by manually mutating the .data member for Variables holding learnable parameters.

`W1.data -= learning_rate * W1.grad.data`

This is not a huge burden for simple optimization algorithms like stochastic gradient descent on a single layer network (such as a multiple linear regession (MLR) model), but in practice we often train neural networks using more sophisticated optimizers like AdaGrad, RMSProp, Adam, etc. More on these optimizers later.

The **optim** package in PyTorch abstracts the idea of an optimization algorithm and provides implementations of commonly used optimization algorithms. Most commonly used methods are already supported, and the interface is general enough, so that more sophisticated ones can be also easily integrated in the future.

The following example brings these two highly scaleable concepts of computational graphs and optimization to life:

* we will use the nn package to define our our linear regression module,
* and we will use optimize the model using the **Adam** algorithm, a variant of stochastic gradient descent. The **Adam** algorithm comes as part of the the **optim** package.

# Building PyTorch Networks using the Sequential API

The **Sequential** class allows us to build a neural network in PyTorch in a high-level quick and modular manner.

* The Sequential class allows us to build PyTorch neural networks on-the-fly without having to build an explicit class.
* This make it much easier to rapidly build networks and allows us to skip over the step where we implement the forward() method. When we use the sequential way of building a PyTorch network, we construct the forward() method implicitly by defining our network's architecture sequentially.
 

Here we use the Sequential() API to build a single-layered neural network (upon closer inspection you see it is a linear regression model) . 

In [ ]:
import torch
from torch.autograd import Variable

# N is batch size; D_in is input dimension;
# D_out is output dimension.
m_rows, D_in, D_out = 64, 1000, 1

torch.manual_seed(0)
# Create random Tensors to hold input and outputs, and wrap them in Variables.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Variables during the backward pass.
X_train = torch.randn((m_rows, D_in), requires_grad=False) #use (m_rows, D_in) or m_rows, D_in
Y_train = torch.randn(m_rows, D_out, requires_grad=False)

# Create random Tensors for weights, and wrap them in Variables.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Variables during the backward pass.
# NOT Need as we are using the nn package
#  W1 = torch.randn(D_in, D_out,  requires_grad=True)

# Use the nn package to define our model and loss function.
# use the sequential API makes things simple
model = torch.nn.Sequential(  #  X_train @ W1
    torch.nn.Linear(D_in, D_out),   # X.matmul(W1)
)
# loss scaffolding layer
loss_fn = torch.nn.MSELoss(size_average=True)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(50):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(X_train)

    # Compute and print loss.
    loss = loss_fn(y_pred, Y_train)
    if epoch % 10 == 0:
         print(f"Epoch:{epoch}, MSE: {loss.item():.9}")

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()
print(model)


# Activation Functions

A single layered  neural network consisting of an input layer (not counted as a layer) and an output layer  is essentially just a linear regression model. A two-layered network consisting for an input layer (not counted as a layer), a hidden layer, and an output layer is still essentially just a linear regression model. Both these networks are just linear transformations of the inputs. To make these networks nonlinear (and enhance their predictive power), we introduce  a non-linear activation function (that acts element-wise on the linear transformed inputs).

 

A popular activation function is the Sigmoid function. It is one of the most widely used non-linear activation function. Sigmoid transforms the values between the range 0 and 1. Here is the mathematical expression for sigmoid:

`f(x) = 1/(1+e^-x)`

``` import numpy as np
def sigmoid_function(x):
    z = (1/(1 + np.exp(-x)))
    return z
```  
    
**ReLU**

The ReLU function is another non-linear activation function that has gained popularity in the deep learning domain. ReLU stands for Rectified Linear Unit. The main advantage of using the ReLU function over other activation functions is that it does not activate all the neurons at the same time. This means that the neurons will only be deactivated if the output of the linear transformation is less than 0. 

```
def relu_function(x):
    if x<0:
        return 0
    else:
        return x
```

`relu_function(7), relu_function(-7). # returns (7, 0) as a result`

For the negative part of the input domain, you will notice that the value is zero.

In PyTorch ReLU is available as a built in layer via  torch.nn.ReLU().


**The following are examples of neural networks consisting of input, hidden, and output layers:**

```    
network1 = nn.Sequential(
    nn.Linear(in_features, out_features), # X.matmul(W1) + b1
    nn.ReLU(), #Nonlinear activation function.  nn.ReLU(X.matmul(W1) + b1) 
    nn.Linear(out_features, 1) # X.matmul(W2) + b2 )
)
```

```
network2 = nn.Sequential(
    nn.Linear(in_features, out_features), # X.matmul(W1) + b1
    nn.Linear(out_features, 1) # X.matmul(W2) + b2 )
)
```

```
network3 = nn.Sequential(
    nn.Linear(in_features, out_features), # X.matmul(W1) + b1
    nn.ReLU(), #Nonlinear activation function.  nn.ReLU(X.matmul(W1) + b1) 
    nn.Linear(in_features, out_features), # X.matmul(W2) + b1
    nn.ReLU(), #Nonlinear activation function.  nn.ReLU(X.matmul(W1) + b1) 
    nn.Linear(out_features, 1) # X.matmul(W3) + b2 )
)
```

# Boston house price regression via Sequential API

Using the starter code below train a neural network (sometimes known as a  multi-layered perceptron) on the Boston house price prediction problem. Experiment with different architectures and report your best one based on the lowest MSE of the test dataset.  Please upload your entire code and the output from your best run here and provide a brief summary and discussion of the best architecture and experimental setup.



In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary  #install it if necessary using !pip install torchsummary 
import torch
#import torchvision
import torch.utils.data
#import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


# is there a GPU availabale. If available use it
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(f"We are working on a {device} device")

# load data
boston = load_boston()

X = boston['data']
y = boston['target']

in_features = X.shape[1]
X.shape

# train validation test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.15, random_state=42)


## Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation) #Transform test set with the same constants
X_test = scaler.transform(X_test) #Transform test set with the same constants

# convert numpy arrays to tensors
X_train_tensor = torch.from_numpy(X_train)
X_validation_tensor = torch.from_numpy(X_validation)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)
y_validation_tensor = torch.from_numpy(y_validation)

# create TensorDataset in PyTorch
boston_train = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
boston_validation = torch.utils.data.TensorDataset(X_validation_tensor, y_validation_tensor)
boston_test = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)
# create dataloader
batch_size = 96
trainloader_boston = torch.utils.data.DataLoader(boston_train, batch_size=batch_size, shuffle=True, num_workers=2)
validloader_boston = torch.utils.data.DataLoader(boston_validation, batch_size=X_test.shape[0], shuffle=False, num_workers=2)
testloader_boston = torch.utils.data.DataLoader(boston_test, batch_size=X_validation.shape[0], shuffle=False, num_workers=2)

#==================================================#
#    Modify the network architecture here          #
#==================================================#
D_in = X_test.shape[1]
#print(D_in)
D_hidden =20
D_out = 1
# Use the nn package to define our model and loss function.
# use the sequential API makes things simple
model = torch.nn.Sequential(    
    torch.nn.Linear(D_in, D_hidden),   # X.matmul(W1)
    nn.ReLU(), #nn.Sigmoid()           # Relu( X.matmul(W1))
    nn.Linear(in_features=D_hidden, out_features=D_out)       # Relu( X.matmul(W1)).matmul(W2)
    
)

#==================================================#
#                                                  #
#          Please don't modify code below here     #
#==================================================#

# MSE loss scaffolding layer
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=0.0001)


print('-'*50)
print('Model:')
print(model)
summary(model, (1, 13))
print('-'*50)


'''
Training Process:
    Load a batch of data.
    Zero the grad.
    Predict the batch of the data through net i.e forward pass.
    Calculate the loss value by predict value and true value.
    Backprop i.e get the gradient with respect to parameters
    Update optimizer i.e gradient update
'''

epochs = range(5)
for epoch in epochs:
    running_loss = 0.0
    count = 0
    for batch, data in enumerate(trainloader_boston):
        inputs, target = data[0].to(device), data[1].to(device)


        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()

        # do forward pass
        output = model(inputs.float())

        # compute loss and gradients
        loss = loss_fn(output, torch.unsqueeze(target.float(), dim=1))
        # get gradients w.r.t to parameters
        loss.backward()

        # perform gradient update
        optimizer.step()

        # loss
        running_loss += loss.item()
        count += 1
        
    print(f"Epoch {epoch+1},Train MSE loss: {np.round(running_loss/count, 3)}")

print('Finished Training')
print('-'*50)

running_loss = 0.0
count = 0
for batch, data in enumerate(validloader_boston):
    inputs, target = data[0].to(device), data[1].to(device)
    # do forward pass
    output = model(inputs.float())

    # compute loss and gradients
    loss = loss_fn(output, torch.unsqueeze(target.float(), dim=1))

    running_loss += loss.item()
    count += 1

print(f" Validation  MSE loss: {np.round(running_loss/count, 3)}")


running_loss = 0.0
count = 0
for batch, data in enumerate(testloader_boston):
    inputs, target = data[0].to(device), data[1].to(device)
    # do forward pass
    output = model(inputs.float())

    # compute loss and gradients
    loss = loss_fn(output, torch.unsqueeze(target.float(), dim=1))

    running_loss += loss.item()
    count += 1
    
print(f" TEST  MSE loss: {np.round(running_loss/count, 3)}")

# Perform Classification on Iris dataset via Sequential API

In [ ]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(f"We are working on a {device} device")

# load data
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target


# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle = True)

## Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) #Transform test set with the same constants

# convert numpy arrays to tensors
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

# create TensorDataset in PyTorch
iris_train = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
iris_test = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# create dataloader
# DataLoader is implemented in PyTorch, which will return an iterator to iterate training data by batch.
batch_size = 20   # batch_size is the size of each batch in which data returned 
trainloader_iris = torch.utils.data.DataLoader(iris_train, batch_size=batch_size, shuffle=True, num_workers=2)
testloader_iris = torch.utils.data.DataLoader(iris_test, batch_size=X_test.shape[0], shuffle=False, num_workers=2)


#==================================================#
#    Modify the network architecture here          #
#==================================================#

D_in = X_test.shape[1]
#print(D_in)
D_hidden = ...
D_out = ...    # Hint: The D_out is the number of class

# Use the nn package to define our model and loss function.
# use the sequential API makes things simple
model = torch.nn.Sequential(
    ...
)

model.to(device)

# use Cross Entropy and SGD optimizer.
loss_fn = ...  #for classfication 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

#==================================================#
#                                                  #
#          Please don't modify code below here     #
#==================================================#

#summary(model, (4, 20))
print('-'*50)
print('Model:')
print(model)
print('-'*50)

epochs = range(5)
'''
Training Process:
    Load a batch of data.
    Zero the grad.
    Predict the batch of the data through net i.e forward pass.
    Calculate the loss value by predict value and true value.
    Backprop i.e get the gradient with respect to parameters
    Update optimizer i.e gradient update
'''
train_losses = {}
train_accuracy = {}
for epoch in epochs:
    running_loss = []
    y_pred = []
    epoch_target = []
    for batch, data in enumerate(trainloader_iris):
        inputs, target = data[0].to(device), data[1].to(device)

        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()

        # do forward pass
        output = model(inputs.float())

        # compute loss and gradients
        loss = loss_fn(output, target)
        # get gradients w.r.t to parameters
        loss.backward()

        # perform gradient update
        optimizer.step()
        
        y_pred.extend(torch.argmax(output, dim=1).tolist())
        epoch_target.extend(target.tolist())
        running_loss.append(loss.item())
        
    epoch_training_loss = np.mean(running_loss)
    train_losses[epoch+1] = epoch_training_loss
    print(f"Epoch {epoch+1}, Training loss: {np.round(epoch_training_loss, 3)}")
    
    #accuracy
    correct = (np.array(y_pred) == np.array(epoch_target))
    accuracy = correct.sum()/ correct.size
    train_accuracy[epoch+1] = accuracy
    print(f"Epoch {epoch+1}, Training accuracy: {np.round(accuracy, 3)}")
    
print(train_losses)

print('Finished Training')
print('-'*50)

test_batch_losses = []
test_y_pred = []
test_target = []
for batch, data in enumerate(testloader_iris):
    inputs, target = data[0].to(device), data[1].to(device)
    # do forward pass
    output = model(inputs.float())

    # compute loss
    loss = loss_fn(output, target)
    
    test_batch_losses.append(loss.item())
    
    test_y_pred.extend(torch.argmax(output, dim=1).tolist())
    test_target.extend(target.tolist())

print(f"Test loss: {np.round(np.mean(test_batch_losses), 3)}")

#accuracy
test_correct = (np.array(test_y_pred) == np.array(test_target))
test_accuracy = test_correct.sum()/ test_correct.size
print(f"Test accuracy: {np.round(test_accuracy, 3)}")


# Boston house price regression via OOP API

In [1]:
from torchsummary import summary  #install it if necessary using !pip install torchsummary 
import torch
#import torchvision
import torch.utils.data
#import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

    
# load data
boston = load_boston()

X = boston['data']
y = boston['target']

in_features = X.shape[1]

# train validation test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

## Scaling
scaler = StandardScaler()
X_train =      scaler.fit_transform(X_train).astype(float)
X_validation = scaler.transform(X_validation).astype(float) #Transform valid set with the same constants
X_test =       scaler.transform(X_test).astype(float)       #Transform test  set with the same constants

# convert numpy arrays to tensors
X_train_tensor = torch.from_numpy(X_train).float()
X_validation_tensor = torch.from_numpy(X_validation).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()
y_validation_tensor = torch.from_numpy(y_validation).float()

# create TensorDataset in PyTorch
train_ds = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
validation_ds = torch.utils.data.TensorDataset(X_validation_tensor, y_validation_tensor)
test_ds = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# create dataloader
batch_size = 96
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(validation_ds, batch_size=X_test.shape[0], shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=X_validation.shape[0], shuffle=False, num_workers=0)

#==================================================#
#    Modify the network architecture here          #
#==================================================#

D_in = X_test.shape[1]
#print(D_in)
D_hidden =20
D_out = 1

# Use the OOP API to define a deep neural network model
#
class BaseModel(nn.Module):
    """Custom module for a simple  regressor"""
    def __init__(self, in_features, size_hidden=10, n_output=1):
        super(BaseModel, self).__init__()
        self.fc1 = torch.nn.Linear(in_features, size_hidden)   # hidden layer
        self.fc2 = torch.nn.Linear(size_hidden, n_output)      # output layer
 
    def forward(self, x):
   
        x = F.relu(self.fc1(x))   # activation function for hidden layer
        x = self.fc2(x)
        return x
        
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(f"We are working on a {device} device")

# create classifier and optimizer objects
model = BaseModel(in_features=D_in, size_hidden = D_hidden, n_output = D_out)
model.to(device) # put on GPU before setting up the optimizer

#==================================================#
#                                                  #
#          Please don't modify code below here     #
#==================================================#

print('-'*50)
print('Model:')
print(model)
summary(model, (1, 13))
print('-'*50)


loss_fn = torch.nn.MSELoss(reduction='mean')
opt = optim.SGD(model.parameters(), lr=0.0001)

loss_history = []
acc_history = []

'''
Training Process:
    Load a batch of data.
    Zero the grad.
    Predict the batch of the data through net i.e forward pass.
    Calculate the loss value by predict value and true value.
    Backprop i.e get the gradient with respect to parameters
    Update optimizer i.e gradient update
'''

def train_epoch(epoch, model, loss_fn, opt, train_loader):
    running_loss = 0.0
    count = 0
    # dataset API gives us pythonic batching 
    for batch_id, data in enumerate(train_loader):
        inputs, target = data[0].to(device), data[1].to(device)        
        # 1:zero the grad, 2:forward pass, 3:calculate loss,  and 4:backprop!
        opt.zero_grad()
        preds = model(inputs) #prediction over the input data

        # compute loss and gradients
        loss = loss_fn(preds, target)    #mean loss for this batch

        loss.backward() #calculate nabla_w
        loss_history.append(loss.item())
        opt.step()  #update W
        #from IPython.core.debugger import Pdb as pdb;    pdb().set_trace() #breakpoint; dont forget to quit
        
        running_loss += loss.item()
        count += 1

    print(f"Train MSE loss: {np.round(running_loss/count, 3)}")



#from IPython.core.debugger import Pdb as pdb;    pdb().set_trace() #breakpoint; dont forget to quit
def evaluate_model(epoch, model, loss_fn, opt, data_loader, tag = "Test"):
    overall_loss = 0.0
    count = 0
    for i,data in enumerate(data_loader):
        inputs, targets = data[0].to(device), data[1].to(device)                
        outputs = model(inputs)      

        loss = loss_fn(outputs, targets)           # compute loss value
        
        overall_loss += (loss.item())  # compute total loss to save to logs
        count += 1
        
        # compute mean loss

    print(f"{tag} MSE loss: {overall_loss/count:.3f}")


for epoch in range(5):
    print(f"Epoch {epoch+1}")
    train_epoch(epoch, model, loss_fn, opt, train_loader)
    evaluate_model(epoch,    model, loss_fn, opt, valid_loader, tag = "Validation")
print("-"*50)
evaluate_model(epoch, model, loss_fn, opt, test_loader, tag="Test")

We are working on a cpu device
--------------------------------------------------
Model:
BaseModel(
  (fc1): Linear(in_features=13, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 20]             280
            Linear-2                 [-1, 1, 1]              21
Total params: 301
Trainable params: 301
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
--------------------------------------------------
Epoch 1
Train MSE loss: 607.063
Validation MSE loss: 632.953
Epoch 2
Train MSE loss: 604.05
Validation MSE loss: 630.960
Epoch 3
Train MSE loss: 605.145
Validatio